In [ ]:
import pandas as pd
from google.cloud import bigquery

# MIMIC-III Data

In [ ]:
!gcloud auth application-default login

In [ ]:
# connecting to the MIMIC-III database
client = bigquery.Client(project='clinical-entity-extraction')

In [ ]:
# Performing query to collate data from the mimic database
q = (
"""
WITH cte1 AS (
    SELECT a.SUBJECT_ID, a.HADM_ID, a.ADMITTIME, a.DIAGNOSIS,
        p.DOB, p.GENDER
    FROM physionet-data.mimiciii_clinical.patients p
    INNER JOIN physionet-data.mimiciii_clinical.admissions a
        ON p.SUBJECT_ID = a.SUBJECT_ID
),
cte2 AS (
    SELECT SUBJECT_ID, HADM_ID, DIAGNOSIS, GENDER, 
        DATE_DIFF(ADMITTIME, DOB, YEAR) AS AGE
    FROM cte1
    WHERE DATE_DIFF(ADMITTIME, DOB, YEAR) < 100
),
cte3 AS (
    SELECT SUBJECT_ID, HADM_ID, CHARTDATE, CATEGORY, TEXT
    FROM physionet-data.mimiciii_notes.noteevents 
    WHERE ISERROR IS NULL 
        AND HADM_ID IS NOT NULL
)
SELECT cte2.SUBJECT_ID, cte2.HADM_ID, cte2.DIAGNOSIS, cte2.GENDER, cte2.AGE,
    cte3.CHARTDATE, cte3.CATEGORY, cte3.TEXT
FROM cte2
INNER JOIN cte3
    ON cte2.SUBJECT_ID = cte3.SUBJECT_ID
        AND cte2.HADM_ID = cte3.HADM_ID
"""
)
df_notes = client.query(q).to_dataframe()

In [ ]:
df_notes.head()

,SUBJECT_ID,HADM_ID,DIAGNOSIS,GENDER,AGE,CHARTDATE,CATEGORY,TEXT
0,5766,101408,OVERDOSE,F,74,2159-09-26,Nursing/other,Nursing Progress Note 7a-7p:\n\nNeuro: Pt cont...
1,4816,160315,FEVER;NEUTROPENIA,M,56,2182-04-18,Nursing/other,NPN 7P-7A\n\nNEURO: PT A/OX3.ABLE TO [** 103**...
2,1356,142815,RESPIRATORY FAILURE,F,75,2187-08-14,Nursing/other,NPN 7P-7A:\n\nNEURO: PT [**Name (NI) **]. SHE ...
3,29264,182651,MITRAL VALVE INSUFFICIENCY\MITRAL VALVE REPLAC...,M,79,2134-10-07,Nursing/other,"Neuro: pt oriented X3, MAE, PERRLA. [**Last Na..."
4,1592,135208,NEWBORN,M,0,2161-02-02,Nursing/other,NPN\n\n\n#1/#6\nInfant remains in RA with ats ...


In [ ]:
# Casting columns into the appropriate format
text_cols = ['DIAGNOSIS', 'GENDER', 'CATEGORY', 'TEXT']
date_cols = ['CHARTDATE']
int_cols = ['SUBJECT_ID', 'HADM_ID', 'AGE']
df_notes[text_cols] = df_notes[text_cols].astype('string')
df_notes[date_cols] = df_notes[date_cols].astype('datetime64[ns]')
df_notes[int_cols] = df_notes[int_cols].astype('int')

In [ ]:
# Saving output to a file
df_notes.to_parquet("/content/drive/MyDrive/Courses/2. Spring 23/BMIN521/Project/data/mimic_notes.parquet")